In [ ]:
import ska_sdp_config
import datetime
import os
import dask
import logging
import sys

TIME_FORMAT = "%Y-%m-%d %H:%M:%S"

LOG = logging.getLogger(__name__)

In [ ]:
!pip uninstall ska-sdp-scripting -y
!pip install -U git+https://gitlab.com/ska-telescope/sdp/ska-sdp-scripting@sp-1667-notebook-script | grep -v "Requirement already"
import ska_sdp_scripting

In [ ]:
NAMESPACE = "dp-dppt-peter"  # add the namespace you want to connect to here
cfg = ska_sdp_config.Config(host=f"ska-sdp-etcd-client.{NAMESPACE}")

In [ ]:
for txn in cfg.txn():
    print(txn.list_processing_blocks())

In [ ]:
image = os.environ['JUPYTER_IMAGE_SPEC']
cpos = image.rindex(':')
image_name = image[:cpos]
image_ver = image[cpos+1:]
script = dict(kind='batch', name=image_name, version=image_ver)
pb = ska_sdp_scripting.ProcessingBlock(None, script, None, cfg)

In [ ]:
for txn in cfg.txn():
    print(txn.list_processing_blocks())

In [ ]:

def inc(x_i):
    time.sleep(2)
    return x_i + 1

def dec(y_i):
    time.sleep(2)
    return y_i - 1

def add(x, y):
    x1 = inc(x)
    y1 = dec(y)
    time.sleep(2)
    z = x1 + y1
    return z

def calc(x, y):
    # This is the top-level function that returns the Dask graph to be executed
    # by the execution engine. It will be called by the EE interface to
    # construct the graph, then the graph will be executed on the Dask
    # deployment by calling its 'compute' method.
    x1 = dask.delayed(inc)(x)
    y1 = dask.delayed(dec)(y)
    z = dask.delayed(add)(x1, y1)
    return z


In [ ]:
helm_values = { "image": os.environ['JUPYTER_IMAGE'] }
helm_values

In [ ]:

n_workers = 3

work_phase = pb.create_phase("Work", [])


In [ ]:
with work_phase:

    # Deploy two instances of a Dask EE
    deploy1 = work_phase.ee_deploy_dask("dask-1", n_workers, calc, (1, 5), helm_values)
    #deploy2 = work_phase.ee_deploy_dask("dask-2", n_workers, calc, (1, 7))

    # Wait until deployments are finished or PB is cancelled
    for txn in work_phase.wait_loop():
        if deploy1.is_finished(txn): # and deploy2.is_finished(txn):
            break
        txn.loop(wait=True)